In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")
np.random.seed(203)
from tqdm import tqdm
import datetime
from collections import Counter
import re

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

from catboost import Pool, CatBoostRegressor

In [ ]:
#!pip install catboost

In [ ]:
data_train = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/train_5CLrC8b/train.csv')
#data_dict = pd.read_csv('/Users/s0c02nj/Desktop/train_5CLrC8b/Data_Dictionary.xlsx')
data_test = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/test.csv')
data_sub = pd.read_csv('/Users/s0c02nj/Desktop/Mahindra/sample_submission.csv')

In [ ]:
# #Train data separating the response
# x_train = data_train.drop('amount_spent_per_room_night_scaled',axis=1)
# y_def = data_train['amount_spent_per_room_night_scaled']

#### Doing Outlier treatement
+ Based on distribution of response variable

In [ ]:
# mu = data_train['amount_spent_per_room_night_scaled'].mean()
# sigma =  np.sqrt(data_train['amount_spent_per_room_night_scaled'].var())

# thres_u = mu+4*sigma
# thres_l = mu-4*sigma

# ind=[]
# for i in tqdm(range(0,len(data_train))):
#     val = data_train['amount_spent_per_room_night_scaled'].iloc[i]
    
#     if val <=thres_u and val >=thres_l :
#         ind.append(1)
#     else :
#         ind.append(0)
        
# data_train['outlier_ind'] = ind


In [ ]:
data_train_new = data_train

In [ ]:
# data_train_new = data_train[data_train['outlier_ind']==1]
# data_train_new = data_train_new.drop(['outlier_ind'],axis=1)

In [ ]:
#Train data separating the response
x_train = data_train_new.drop('amount_spent_per_room_night_scaled',axis=1)
y_def =  data_train_new['amount_spent_per_room_night_scaled']

In [ ]:
#Test data and train data merged for pre-processing
x_comb = pd.concat([x_train,data_test],sort=False)

Missing value current
+ season_holidayed_code        149
+ state_code_residence        7024

Replacing Missing values with 'unknown' token for now

In [ ]:
#x_comb = x_comb.replace(np.nan, 0.0, regex=True)
#x_comb.head()

In [ ]:
x_comb['season_holidayed_code'].fillna(x_comb['season_holidayed_code'].mode()[0], inplace=True)
x_comb['state_code_residence'].fillna(x_comb['state_code_residence'].mode()[0], inplace=True)

Time Related features engineering

In [ ]:
from datetime import timedelta, date
x_comb['booking_date'] =  pd.to_datetime(x_comb['booking_date'], format='%d/%m/%y')
x_comb['checkin_date'] =  pd.to_datetime(x_comb['checkin_date'], format= '%d/%m/%y')
x_comb['checkout_date'] = pd.to_datetime(x_comb['checkout_date'],format= '%d/%m/%y')

In [ ]:
#Duration stay
x_comb['Duration_stay'] = x_comb['checkout_date'] - x_comb['checkin_date']
x_comb['Duration_stay'] = x_comb['Duration_stay'].apply(lambda x:x.days)

In [ ]:
x_comb.shape

In [ ]:
#Date-time year for all the time variables
cols =['booking_date','checkin_date','checkout_date']


for i in tqdm(cols):
    x_comb[i +' '+'month'] = x_comb[i].dt.month
    x_comb[i +' '+'week'] =  x_comb[i].dt.week
    x_comb[i +' '+'year'] =  x_comb[i].dt.year
    x_comb[i +' '+'day'] =   x_comb[i].dt.day
        

In [ ]:
x_comb = x_comb.drop(['booking_date','checkin_date','checkout_date'],axis=1)

In [ ]:
x_comb.columns

In [ ]:
total_cols = ['channel_code', 'main_product_code', 'numberofadults',
       'numberofchildren',  'resort_region_code',
       'resort_type_code', 'room_type_booked_code', 'roomnights',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'total_pax', 'member_age_buckets', 'booking_type_code','memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'Duration_stay', 'booking_date month', 'booking_date week',
       'booking_date year', 'booking_date day', 'checkin_date month',
       'checkin_date week', 'checkin_date year', 'checkin_date day',
       'checkout_date month', 'checkout_date week', 'checkout_date year',
       'checkout_date day']

In [ ]:
cat_cols = ['channel_code', 'main_product_code',
        'resort_region_code',
       'resort_type_code', 'room_type_booked_code',
       'season_holidayed_code', 'state_code_residence', 'state_code_resort',
       'member_age_buckets', 'booking_type_code','memberid',
       'cluster_code', 'reservationstatusid_code', 'resort_id',
       'booking_date month', 'booking_date week',
       'booking_date year', 'checkin_date month',
       'checkin_date week', 'checkin_date year',
       'checkout_date month', 'checkout_date week', 'checkout_date year']

In [ ]:
x_comb['roomnights'] = x_comb['roomnights'].replace(-45, 45)

In [ ]:
for col in tqdm(cat_cols):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col])

In [ ]:
x_comb = x_comb.drop(['reservation_id','persontravellingid'],axis=1)

In [ ]:
# new_cols = ['resort_region_code','reservation_id',
#        'resort_type_code', 'room_type_booked_code',
#        'season_holidayed_code', 'state_code_residence', 'state_code_resort',
#        'total_pax', 'member_age_buckets', 'booking_type_code', 'memberid',
#        'cluster_code', 'reservationstatusid_code', 'resort_id',
#        'Duration_stay']

# new_ls=[]
# for col1 in tqdm(new_cols):
#     for col2 in new_cols :
#         if col1 != col2:
#             new_ls.append(col1+col2)
#             x_comb[col1+col2] = x_comb[col1]*x_comb[col2]
            
# for col in tqdm(new_ls):
#     le = LabelEncoder()
#     x_comb[col] = le.fit_transform(x_comb[col])

#total_cat_cols = cat_cols + new_ls

In [ ]:
train_x = x_comb[0:341424]
test_x =  x_comb[341424:]

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_def,test_size=0.2,random_state=0)

In [ ]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostRegressor(iterations=1300, 
                           learning_rate=0.03, 
                           l2_leaf_reg=4, 
                           depth=12, 
                           eval_metric='RMSE',
                           use_best_model=True,
                           random_seed=42)

In [ ]:
model.fit(x1_train,y1_train,cat_features= cat_cols, eval_set=(x1_val,y1_val))

In [ ]:
#pred_trans = model.predict(test_x)
#pred = np.expm1(pred_trans)

In [ ]:
pred_trans = model.predict(test_x)
pred = (1.0/pred_trans)-1


In [ ]:
pred = model.predict(test_x)

In [ ]:
data_sub['amount_spent_per_room_night_scaled']=pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/Mahindra/Sub_cat6.csv',index=False)